In [4]:
import pandas as pd
from proxy_finder import find_proxies

In [28]:
litt_SNPs = ["rs12566888",
    "rs2363910",
    "rs7940646",
    "rs12566888",
    "rs4311994",
    "rs10761741",
    "rs1671152",
    "rs12041331",
    "rs12041331",
    "rs12041331",
    "rs11924165",
    "rs10883735",
    "rs11202221",
    "rs6566765",
    "rs9889955",
    "rs10886430",
    "rs12566888",
    "rs7624918",
    "rs1613662",
    "rs773902",
    "rs10886430",
    "rs12137738",
    "rs142001088",
    "rs12041331",
    "rs12041331",
    "rs12041331",
    "rs1175170",
    "rs112157462",
    "rs58250884",
    "rs185159562",
    "rs138028657",
    "rs7097060",
    "rs183146849",
    "rs140148392",
    "rs61974290",
    "rs575524466",
    "rs1207486385",
    "rs1207486385",
    "rs138845468",
    "rs542707094",
    "rs61751937",
    "rs143238312",
    "rs150291014",
    "rs763817905",
    "rs61752770"]

epinephrine_not_gene_based = [
    "rs12566888",
    "rs4311994",
    "rs10761741",
    "rs12041331",
    "rs12137738",
    "rs12041331",
    "rs1175170",
    "rs58250884",
    "rs185159562",
    "rs7097060",
    "rs61974290",
    "rs1207486385",
]

gene_based = [
    "rs61751937",
    "rs143238312",
    "rs150291014",
    "rs763817905",
    "rs61752770",
]

epinephrine_or_gene_based = list(set(epinephrine_not_gene_based + gene_based))

In [6]:
#agonist = "ADP_META"
agonists = ["ADP_META","CRP","PAR1","PAR4"]

pfcs = []
for ago in agonists:
    pfc = pd.read_csv("pfc_results/%s.tsv" % ago,sep="\t",usecols=["CHR","POS","RSID","MLOG10P","AAF"])
    #pfc = pfc.loc[pfc.AAF > 0.01]
    #pfc = pfc.sort_values("MLOG10P",ascending=False)
    pfc["agonist"] = ago
    pfcs.append(pfc)
pfc = pd.concat(pfcs,axis=0)
pfc_best = pfc.sort_values("MLOG10P").groupby(["CHR","POS"])[["MLOG10P","RSID"]].last().reset_index()
pfc_snps = pfc_best[["CHR","POS"]].astype(str).agg("_".join,axis=1).tolist()
pfc.head()

,RSID,CHR,POS,AAF,MLOG10P,agonist
0,rs3131972,1,752721,0.83720,0.03656,ADP_META
1,rs3115860,1,753405,0.86180,0.04916,ADP_META
2,rs2073813,1,753541,0.13770,0.04938,ADP_META
3,rs12184312,1,754063,0.03512,0.32430,ADP_META
4,rs12184325,1,754105,0.03496,0.33750,ADP_META


In [10]:
missing_rsIDs = list(set(litt_SNPs).difference(set(pfc_snps)))

In [16]:
missing_rsIDs[:4]

['rs10886430', 'rs2363910', 'rs575524466', 'rs12137738']

In [13]:
valid_coords = ("chr" + pfc_best["CHR"].astype(str) + ":" + pfc_best["POS"].astype(str)).unique().tolist()
    
proxies = find_proxies(
    missing_rsIDs,
    proxies_json_path="litterature_proxies.json",
    valid_coords = valid_coords,
    LDLink_API_token = "14ad36e8238c",
)

Loading pre-existing proxies
3 (16) / 36
Sending request for rs12137738...
Found proxy for rs12137738 : rs12137738
6 (17) / 36
rs763817905 is not in reference panel (NCBI)
7 (18) / 36
Sending request for rs7097060...
Found proxy for rs7097060 : rs7097060
8 (19) / 36
Sending request for rs11924165...
{
  "error": "Variant is monoallelic in the chosen population(s)."
}
9 (20) / 36
rs1207486385 is not in reference panel (NCBI)
12 (21) / 36
Sending request for rs6566765...
Found proxy for rs6566765 : rs6566765
13 (22) / 36
Sending request for rs61751937...
Found proxy for rs61751937 : rs61751937
14 (23) / 36
Sending request for rs61974290...
Found proxy for rs61974290 : rs61974290
15 (24) / 36
Sending request for rs112157462...
Found proxy for rs112157462 : rs112157462
16 (25) / 36
Sending request for rs9889955...
Found proxy for rs9889955 : rs9889955
17 (26) / 36
Sending request for rs7940646...
Found proxy for rs7940646 : rs7940646
20 (27) / 36
rs150291014 is not in reference panel (NCBI

In [26]:
matched_rsIDs = [m for m in missing_rsIDs if "error" not in proxies[m]]
print("%d matched IDs" % len(matched_rsIDs))

28 matched IDs


In [27]:
pfc_best["valid_coords"] = valid_coords
pfc_best["represents_SNP"] = None

In [29]:
for m in matched_rsIDs:
    if m not in epinephrine_or_gene_based:
        pfc_best.loc[pfc_best.valid_coords == proxies[m]["coords"],"represents_SNP"] = m

In [30]:
pfc_best.loc[pfc_best.RSID.isin(set(litt_SNPs).difference(epinephrine_or_gene_based)),"represents_SNP"] = pfc_best.loc[pfc_best.RSID.isin(litt_SNPs),"RSID"]

In [32]:
pfc_best.loc[~pfc_best.represents_SNP.isnull()]

,CHR,POS,MLOG10P,RSID,valid_coords,represents_SNP
175362,1,67594324,0.7567,rs142001088,chr1:67594324,rs142001088
1432744,3,56901292,10.2400,rs7624918,chr3:56901292,rs7624918
2462432,5,19110102,0.4164,rs112157462,chr5:19110102,rs112157462
3879702,7,155760863,1.0790,rs2363910,chr7:155760863,rs2363910
4897005,10,77250649,0.4000,rs138028657,chr10:77250649,rs138028657
4932914,10,88602314,0.8186,rs11202221,chr10:88602314,rs11202221
4972853,10,104308446,0.3669,rs10883735,chr10:104308446,rs10883735
5018254,10,121010256,39.5600,rs10886430,chr10:121010256,rs10886430
5108090,11,10669228,0.3926,rs7940646,chr11:10669228,rs7940646
5331656,11,91918231,0.4330,rs183146849,chr11:91918231,rs183146849


In [33]:
# Including even the gene-based ones

In [34]:
pfc_best["valid_coords"] = valid_coords
pfc_best["represents_SNP"] = None

for m in matched_rsIDs:
    pfc_best.loc[pfc_best.valid_coords == proxies[m]["coords"],"represents_SNP"] = m
    
pfc_best.loc[pfc_best.RSID.isin(litt_SNPs),"represents_SNP"] = pfc_best.loc[pfc_best.RSID.isin(litt_SNPs),"RSID"]

pfc_best.loc[~pfc_best.represents_SNP.isnull()]

,CHR,POS,MLOG10P,RSID,valid_coords,represents_SNP
60323,1,20894442,0.9288,rs12137738,chr1:20894442,rs12137738
175362,1,67594324,0.7567,rs142001088,chr1:67594324,rs142001088
337879,1,156869047,7.4970,rs12566888,chr1:156869047,rs12566888
337883,1,156869714,7.2160,rs12041331,chr1:156869714,rs12041331
437799,1,192164010,1.4990,rs1175170,chr1:192164010,rs1175170
1432744,3,56901292,10.2400,rs7624918,chr3:56901292,rs7624918
2462432,5,19110102,0.4164,rs112157462,chr5:19110102,rs112157462
3285780,6,122243017,1.3790,rs58250884,chr6:122243017,rs58250884
3879702,7,155760863,1.0790,rs2363910,chr7:155760863,rs2363910
4592996,9,113312231,5.0840,rs61751937,chr9:113312231,rs61751937
